# Newton-Raphson fractal animation notebook

In [ ]:
import os
import sys
import numpy as np
from tqdm import tqdm
from itertools import product
from joblib import Parallel, delayed

import matplotlib.pyplot as plt

from polynomial import Polynomial

from newton_engine import get_NR_colors
from newton_plots import NR_fractal_mx

In [ ]:
N = 1024
n_steps = 50
n_frames = 90
ffmt = 'png'
fdpi = 100  

outdir = './out/frames-N{}-ns{}/'.format(N, n_steps)
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [ ]:
# Polynomial to calculate
P = Polynomial(c=[1,0,0,1,-1,1])

# Predefined limits
gl_s = ((-1.5,1.5),(-1.5,1.5))
gl_e = ((-1.0092734515,-1.0092734495), (0.1473217440, 0.1473217460))

In [ ]:
def NR_fractal_get_frames(gl_s, gl_e, n_frames=50):
  
  v_s = np.array(list(product(*gl_s)))
  v_e = np.array(list(product(*gl_e)))

  sg = np.sign(v_s - v_e)
  f_s = v_s - v_e
  f_e = np.zeros_like(v_s) + 1e-15 * sg

  v_coords = np.geomspace(f_s, f_e, n_frames) + v_e
  
  return v_coords

def NR_fractal_get_grid_lims(v_coords):
  
  # Shorten variable names
  v1, v2 = v_coords[:,0], v_coords[:,3]
  
  grid_lims =\
  tuple(
    tuple(
      ( (xmin, xmax),(ymin, ymax) )
    )
    for xmin, xmax, ymin, ymax in zip(v1[:,0], v2[:,0], v1[:,1], v2[:,1])
  )
  
  return grid_lims

In [ ]:
def create_frame(i, P,
                 N, n_steps, gl,
                 outdir, ffmt, fdpi):
  X_n, _, _, _= NR_fractal_mx(P,
                              N=N, n_steps=n_steps,
                              grid_lim_x=gl[0],
                              grid_lim_y=gl[1])
  X = get_NR_colors(P, X_n).reshape((N, N, 4))

  fname = 'nrfractal_anim_frame-{:04d}.'.format(i)
  plt.imsave(fname=(outdir + fname + ffmt),
             arr=X, vmin=0, vmax=1,
             dpi=fdpi)

In [ ]:
# Get the extents of the frames
v_coords = NR_fractal_get_frames(gl_s, gl_e, n_frames=n_frames)
grid_lims = NR_fractal_get_grid_lims(v_coords)
#del v_coords

In [ ]:
%%time
_ = Parallel(n_jobs=4)(delayed(create_frame)(i, P,
                                             N, n_steps, gl,
                                             outdir, ffmt, fdpi) for i, gl in enumerate(tqdm(grid_lims)));

## Append frames

In [ ]:
import imageio
from datetime import datetime

In [ ]:
def create_anim(N : int, n_steps : int):
  dir_ = 'frames-N{}-ns{}/'.format(N, n_steps)
  name = 'NR_fractal-N{}-ns{}'.format(N, n_steps)

  path = './out/' + dir_
  outdir = './out/' + dir_ + 'anim/'
  if not os.path.exists(outdir):
    os.makedirs(outdir)

  # Gather path to appropriate images to concatenate
  print('[{0}] Reading in list of images to concatenate'.format(datetime.now().isoformat()), file=sys.stderr)
  file_list = []
  for file in os.listdir(path):
    if 'nrfractal' in file:
      complete_path = path + file
      file_list.append(complete_path)
  # Sort filenames
  file_list.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))

  # Define writer and create video
  writer = imageio.get_writer(outdir + '{}.mp4'.format(name),
                              codec='h264', bitrate='3500k',
                              format='mp4', fps=30)

  for i, im in enumerate(file_list):
    text = '[{0}] Appending image {1}...'.format(datetime.now().isoformat(), i)
    print(text, file=sys.stderr)
    writer.append_data(imageio.imread(im))
  writer.close()

In [ ]:
create_anim(N=N, n_steps=n_steps)